# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,palikir - national government center,6.9248,158.1611,28.41,79,75,3.09,FM,1711999502
1,1,oriximina,-1.7656,-55.8661,31.71,61,95,1.31,BR,1711964656
2,2,la passe,45.5549,-0.8967,11.66,76,50,9.45,FR,1711950188
3,3,jamestown,42.0970,-79.2353,14.45,53,0,5.14,US,1711969115
4,4,atafu village,-8.5421,-172.5159,29.35,74,16,5.71,TK,1712079181


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng","Lat", geo = True,
    tiles = "OSM",
    width = 900,
    height = 500,
    size = "Humidity",
    color = "City")

# Display the map
hvplot.save(map_plot,"output_data/Fig5.html")

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
temp_df = city_data_df.loc[(city_data_df['Max Temp'] > 20)  & (city_data_df['Max Temp'] < 26)]
wind_df = temp_df.loc[temp_df['Wind Speed'] < 3]
vacation_df = wind_df.loc[wind_df['Cloudiness'] < 5]

# Drop any rows with null values
vacation_df = vacation_df.dropna(how = "any")

# Display sample data
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
80,80,newman,37.3138,-121.0208,20.13,61,0,2.68,US,1711979315
137,137,port blair,11.6667,92.7500,24.02,94,2,1.03,IN,1712015090
227,227,humahuaca,-23.2054,-65.3505,22.01,29,4,1.71,AR,1711967372
302,302,salavan,15.7167,106.4167,25.53,58,0,1.07,LA,1712011718
414,414,myeik,12.4333,98.6000,25.73,74,0,1.39,MM,1712013669
534,534,taungoo,18.9333,96.4333,25.29,53,0,1.04,MM,1712014035
586,586,mormugao,15.4000,73.8000,25.55,77,0,2.68,IN,1712019549
612,612,mulayjah,27.2710,48.4242,24.06,37,0,1.63,SA,1711938994


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = vacation_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotels_df = hotel_df.copy()
hotels_df.loc[:,"Hotel Name"] = ""
# Display sample data
hotels_df

,City,Country,Lat,Lng,Humidity,Hotel Name
80,newman,US,37.3138,-121.0208,61,
137,port blair,IN,11.6667,92.7500,94,
227,humahuaca,AR,-23.2054,-65.3505,29,
302,salavan,LA,15.7167,106.4167,58,
414,myeik,MM,12.4333,98.6000,74,
534,taungoo,MM,18.9333,96.4333,53,
586,mormugao,IN,15.4000,73.8000,77,
612,mulayjah,SA,27.2710,48.4242,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {"categories": categories,
          "apiKey": geoapify_key,
          "limit": 1}
    


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params).json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotels_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotels_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel Name']}")

# Display sample data
hotels_df

Starting hotel search


newman - nearest hotel: No hotel found
port blair - nearest hotel: Andaman Ocean Breeze
humahuaca - nearest hotel: Kuntur Wasi
salavan - nearest hotel: Phengthavixay
myeik - nearest hotel: Green Eyes Hotel
taungoo - nearest hotel: Mothers Guesthouse
mormugao - nearest hotel: Hotel Rukhmini
mulayjah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
80,newman,US,37.3138,-121.0208,61,No hotel found
137,port blair,IN,11.6667,92.7500,94,Andaman Ocean Breeze
227,humahuaca,AR,-23.2054,-65.3505,29,Kuntur Wasi
302,salavan,LA,15.7167,106.4167,58,Phengthavixay
414,myeik,MM,12.4333,98.6000,74,Green Eyes Hotel
534,taungoo,MM,18.9333,96.4333,53,Mothers Guesthouse
586,mormugao,IN,15.4000,73.8000,77,Hotel Rukhmini
612,mulayjah,SA,27.2710,48.4242,37,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
vacation_plot = hotels_df.hvplot.points("Lng", "Lat", geo = True,
    tiles = "OSM",
    width = 900,
    height = 500,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    hover_cols = ["Hotel Name","Country"])
# Display the map

hvplot.save(vacation_plot,"output_data/Fig6.html")

vacation_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)